In [250]:
import pandas as pd
import numpy as np

# 선수 경기 데이터

## 데이터 전처리

In [251]:
df=pd.read_csv("C:\\Users\\066\\Desktop\\FlyAI_Project\\Data\\RawData\\LCK_2023_SUMMER.csv")

In [252]:
#경기 번호 추가
new_column_values = []
for i in range(len(df)):
    value = (i // 10) + 1  # 5개씩 끊어서 1, 2, 3으로 적용
    new_column_values.append(value)

df['gameidx'] = new_column_values

last_column = df.pop(df.columns[-1])  # 마지막 열 추출
df.insert(0, last_column.name, last_column) #마지막 열 제일 앞으로

In [253]:
df.insert(8, 'KDA 0.5', (df['Kills']+df['Assists'])/(df['Deaths']+0.5)) #KDA값 변경
df=df.drop(['KDA','Solo kills','Shutdown bounty collected','Shutdown bounty lost'],axis=1) #안쓰는 열 드롭

#컬럼명 변경
col_list=['gameidx']
username=''
with open('C:\\Users\\066\\Desktop\\FlyAI_Project\\API\\LeagueofLegends\\APIcols.txt', 'r', encoding='utf-8') as file:
    for line in file:
        col_list.append(line.strip())
df.columns=col_list


In [254]:
#결측치 처리
df=df.dropna(subset=['totalMinionsKilled','visionScore'])
df=df.fillna(0)

## 데이터 넣기

In [255]:
import pymysql

# MySQL 연결 정보 설정
hostname='cdymysql.mysql.database.azure.com'
username='cdy'
password='Pa12341234!@#$'
database='testdb'

# MySQL 데이터베이스 연결
connection = pymysql.connect(user=username, password=password, host=hostname, port=3306, database=database, ssl_ca="DigiCertGlobalRootCA.crt.pem", ssl_disabled=False)
# 커서 생성
cursor = connection.cursor()



In [256]:
# 테이블 컬럼 데이터 생성 및 테이블 생성 쿼리
table_name='summer'
create_table_query = f"CREATE TABLE {table_name} ("

for col_name, col_dtype in zip(df.columns, df.dtypes):
    if col_dtype == np.int64:
        mysql_dtype = "INT"
    elif col_dtype == np.float64:
        mysql_dtype = "FLOAT"
    else:
        mysql_dtype = "VARCHAR(255)"
    create_table_query += f"`{col_name}` {mysql_dtype}, "

create_table_query = create_table_query.rstrip(', ') + ", PRIMARY KEY (gameidx, Player))"
print(create_table_query)
cursor.execute(create_table_query)

CREATE TABLE summer (`gameidx` INT, `Player` VARCHAR(255), `championName` VARCHAR(255), `teamPosition` VARCHAR(255), `champLevel` INT, `kills` INT, `deaths` INT, `assists` INT, `KDA 0.5` FLOAT, `totalMinionsKilled` INT, `totalAllyJungleMinionsKilled` FLOAT, `totalEnemyJungleMinionsKilled` FLOAT, `CSM` FLOAT, `goldEarned` INT, `GPM` INT, `GOLD ratio` VARCHAR(255), `visionScore` FLOAT, `wardsPlaced` FLOAT, `wardsKilled` FLOAT, `visionWardsBoughtInGame` FLOAT, `detectorWardsPlaced` INT, `VSPM` FLOAT, `WPM` FLOAT, `VWPM` FLOAT, `WCPM` FLOAT, `VS ratio` VARCHAR(255), `totalDamageDealtToChampions` INT, `physicalDamageDealtToChampions` INT, `magicDamageDealtToChampions` INT, `trueDamageDealtToChampions` INT, `DPM` INT, `DMG ratio` VARCHAR(255), `K+A Per Minute` FLOAT, `KP ratio` VARCHAR(255), `doubleKills` FLOAT, `tripleKills` FLOAT, `quadraKills` FLOAT, `pentaKills` FLOAT, `GD@15` FLOAT, `CSD@15` FLOAT, `XPD@15` FLOAT, `LVLD@15` FLOAT, `objectivesStolen` INT, `damageDealtToTurrets` FLOAT, `d

0

In [257]:
# 데이터프레임 데이터를 테이블에 삽입
insert_query = f"INSERT INTO {table_name} (`{'`, `'.join(df.columns)}`) VALUES ({', '.join(['%s']*len(df.columns))})"
values = df.values.tolist()
values_tuple = [tuple(row) for row in values]
print(insert_query)
cursor.executemany(insert_query, values_tuple)  # 여기서 values 전체를 전달해야 합니다
#cursor.execute(insert_query,values[0])
# 변경사항을 커밋하여 저장
connection.commit()

# 연결 종료
#cursor.close()
#connection.close()

INSERT INTO summer (`gameidx`, `Player`, `championName`, `teamPosition`, `champLevel`, `kills`, `deaths`, `assists`, `KDA 0.5`, `totalMinionsKilled`, `totalAllyJungleMinionsKilled`, `totalEnemyJungleMinionsKilled`, `CSM`, `goldEarned`, `GPM`, `GOLD ratio`, `visionScore`, `wardsPlaced`, `wardsKilled`, `visionWardsBoughtInGame`, `detectorWardsPlaced`, `VSPM`, `WPM`, `VWPM`, `WCPM`, `VS ratio`, `totalDamageDealtToChampions`, `physicalDamageDealtToChampions`, `magicDamageDealtToChampions`, `trueDamageDealtToChampions`, `DPM`, `DMG ratio`, `K+A Per Minute`, `KP ratio`, `doubleKills`, `tripleKills`, `quadraKills`, `pentaKills`, `GD@15`, `CSD@15`, `XPD@15`, `LVLD@15`, `objectivesStolen`, `damageDealtToTurrets`, `damageDealtToBuildings`, `totalHeal`, `totalHealsOnTeammates`, `damageSelfMitigated`, `totalDamageShieldedOnTeammates`, `timeCCingOthers`, `totalTimeCCDealt`, `trueDamageTaken`, `totalTimeSpentDead`, `consumablesPurchased`, `itemsPurchased`) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s,

# 장비데이터

##데이터 전처리

In [258]:
df=pd.read_csv("C:\\Users\\066\\Desktop\\FlyAI_Project\\Data\\RawData\\Equipment - Equipment.csv")

df=df[['Player','Mouse','Mousepad','Keyboard','Headset','Monitor']]
df = df.where(pd.notnull(df), None) #결측치 이름 변경

In [259]:
table_name='equip'
create_table_query = f"CREATE TABLE {table_name} ("

for col_name, col_dtype in zip(df.columns, df.dtypes):
    if col_dtype == np.int64:
        mysql_dtype = "INT"
    elif col_dtype == np.float64:
        mysql_dtype = "FLOAT"
    else:
        mysql_dtype = "VARCHAR(255)"
    create_table_query += f"`{col_name}` {mysql_dtype}, "

create_table_query = create_table_query.rstrip(', ') + ", PRIMARY KEY (Player))"
print(create_table_query)
cursor.execute(create_table_query)

CREATE TABLE equip (`Player` VARCHAR(255), `Mouse` VARCHAR(255), `Mousepad` VARCHAR(255), `Keyboard` VARCHAR(255), `Headset` VARCHAR(255), `Monitor` VARCHAR(255), PRIMARY KEY (Player))


0

In [260]:
# 데이터프레임 데이터를 테이블에 삽입
insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['%s']*len(df.columns))})"
values = df.values.tolist()
values_tuple = [tuple(row) for row in values]
cursor.executemany(insert_query, values_tuple)  # 여기서 values 전체를 전달해야 합니다
#cursor.execute(insert_query,values[0])
# 변경사항을 커밋하여 저장
connection.commit()

# 연결 종료
cursor.close()
connection.close()